In [1]:
!nvidia-smi

Tue Dec 12 11:40:09 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070        Off | 00000000:01:00.0  On |                  N/A |
| 44%   55C    P8              24W / 220W |    168MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# preparar algunas funciones para el programa

In [2]:
import zipfile, os

def descomprimir_zip(archivo_zip, directorio_destino):
    try:
        with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
            zip_ref.extractall(directorio_destino)
        print("¡Archivo ZIP descomprimido correctamente!")
    except zipfile.BadZipFile:
        print("Error: El archivo proporcionado no es un archivo ZIP válido.")
    except Exception as e:
        print(f"Error al descomprimir el archivo ZIP: {e}")




#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [3]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO
from IPython.display import display, Image

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 7966MiB)
Setup complete ✅ (6 CPUs, 31.2 GB RAM, 75.4/467.9 GB disk)


# setear los directorios de trabajo

In [4]:
%cd /content

HOME = os.getcwd()

!mkdir {HOME}/datasets
%cd {HOME}/datasets


print(HOME)

/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/content
mkdir: cannot create directory ‘/content/datasets’: File exists
/content/datasets
/content


# elegir un dataset para entrenar

## ver los datasets disponibles para entrenar

In [5]:
!ls /datasets

json_to_yolo.py  orient.zip


## elegir un dataset

In [6]:
NOMBRE_DATASET_USAR = "orient"

## descomprimir el dataset para realizar el entrenamiento

In [7]:


%cp /datasets/{NOMBRE_DATASET_USAR}.zip /content/dataset.zip


class dataset:
    location = "/content"


# Uso del ejemplo:
archivo_zip = '/content/dataset.zip'  # Ruta y nombre del archivo ZIP que quieres descomprimir
directorio_destino = '/content'  # Ruta del directorio donde se extraerán los archivos

descomprimir_zip(archivo_zip, directorio_destino)

¡Archivo ZIP descomprimido correctamente!


# entrenar

In [8]:
BATCH = 6 # ajustar hasta ocupar el maximo de la GPU posible

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8m.pt data={dataset.location}/data.yaml epochs=50 imgsz=800 plots=True batch={BATCH} save=true

/content
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 7966MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/content/data.yaml, epochs=50, patience=50, batch=6, imgsz=800, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simp

In [ ]:
TRAIN_NUMBER="train2"

# opcional

## ver resultados del entrenamiento [opcional]

In [ ]:
!ls {HOME}/runs/detect/{TRAIN_NUMBER}/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/{TRAIN_NUMBER}/confusion_matrix.png', width=1100)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/{TRAIN_NUMBER}/results.png', width=1100)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/{TRAIN_NUMBER}/val_batch0_pred.jpg', width=1100)

In [ ]:
Image(filename=f'{HOME}/runs/detect/{TRAIN_NUMBER}/val_batch2_pred.jpg', width=1100)

## validar el entrenamiento [opcional]

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/{TRAIN_NUMBER}/weights/best.pt data={dataset.location}/data.yaml

## realizar una prueba del entrenamiento realizado [opcional]

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/{TRAIN_NUMBER}/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

# Guardar los resultados [obligatorio]

In [ ]:
import shutil
shutil.make_archive('train','zip', f"{HOME}/runs/detect/{TRAIN_NUMBER}/")
%cp /content/train.zip /datasets/{NOMBRE_DATASET_USAR}_train.zip

print("Resultados guardados en:")
print(f"http://192.168.1.35:4443/files/{NOMBRE_DATASET_USAR}_train.zip")